<a href="https://colab.research.google.com/github/Joe270585/Reducao_Dimensionalidade_Imagens/blob/main/Reducao_Dimensionalidade_Imagem_ppm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Carregar a imagem

### Subtask:
Como esse código é totalmente sem uso de bibliotecas para tratamento de imagem, as imagens carregadas nele devem ser do formato .ppm.


In [ ]:
# Criando função para carregar imagem
def load_img(filepath):

    try:
        with open(filepath, 'rb') as f:
            # Verificando se formato está correto
            magic_number = f.readline().strip()
            if magic_number != b'P6':
                print("Erro: Formato da imagem não suportado, o formato deve ser .ppm:")
                return None

            # Ler comentários (ignorar linhas que começam com #)
            while True:
                line = f.readline().strip()
                if not line.startswith(b'#'):
                    break

            # Ler dimensões (largura e altura)
            dimensions = line.split()
            if len(dimensions) != 2:
                print(f"Erro: Dimensão Inválida: {line}")
                return None
            width = int(dimensions[0])
            height = int(dimensions[1])

            # Ler valor máximo de cor
            max_color_value_line = f.readline().strip()
            max_color_value = int(max_color_value_line)

           # Ler dados de pixel
           # Para P6, cada pixel tem 3 bytes (R, G, B)
            pixel_data_size = width * height * 3
            pixel_data_bytes = f.read(pixel_data_size)

            if len(pixel_data_bytes) != pixel_data_size:
                 print("Erro: Dados de pixel incompleto.")
                 return None

            # Organizar dados de pixel numa lista de listas
            image_data = []
            for i in range(height):
                row = []
                for j in range(width):
                    pixel_start_index = (i * width + j) * 3
                    r = pixel_data_bytes[pixel_start_index]
                    g = pixel_data_bytes[pixel_start_index + 1]
                    b = pixel_data_bytes[pixel_start_index + 2]
                    row.append([r, g, b])
                image_data.append(row)

            return image_data, width, height, max_color_value

    except FileNotFoundError:
        print(f"Erro: Arquivo não encontrado em {filepath}")
        return None
    except Exception as e:
        print(f"Ocorreu um erro: {e}")
        return None



## Exibir a imagem resultante

### Subtask:
Encontrar uma maneira de exibir os dados de pixel resultantes sem usar bibliotecas de exibição de imagem. Isso pode ser um desafio e pode exigir a criação de um arquivo de imagem simples ou a representação dos pixels de alguma outra forma.


## Implementar conversão para tons de cinza

### Subtask:
Desenvolver código para converter os dados de pixel coloridos (geralmente representados por valores RGB) para tons de cinza. Isso pode ser feito usando fórmulas de luminosidade ou média simples.


In [ ]:
def converte_cinza(image_data, max_color_value):

    # Converte dados de imagem colorida (RGB) para escala de cinza.
    img_cinza = []
    for row in image_data:
        cinza_row = []
        for pixel in row:
            r, g, b = pixel
            # Usando o método de luminosidade para conversão em escala de cinza
            cinza_value = int(0.2989 * r + 0.5870 * g + 0.1140 * b)

            # Garantir que o valor em escala de cinza está dentro do intervalo válido
            cinza_value = max(0, min(cinza_value, max_color_value))

            cinza_row.append(cinza_value)
        img_cinza.append(cinza_row)
    return img_cinza




## Implementar conversão para preto e branco

### Subtask:
Desenvolver código para converter os dados de pixel coloridos ou em tons de cinza para preto e branco. Isso geralmente envolve a aplicação de um limiar (threshold) para decidir se um pixel se torna preto ou branco.


In [ ]:
def converte_p_b(image_data, max_color_value, threshold=None):

    # Determinar se a entrada é colorida ou em escala de cinza
    is_color = any(isinstance(pixel, list) for row in image_data for pixel in row)

    if is_color:
        # Converter a cor para escala de cinza primeiro
        img_cinza = converte_cinza(image_data, max_color_value)
    else:
        # A entrada já está em escala de cinza
        img_cinza = image_data

    # Use um limiar padrão se nenhum for fornecido
    if threshold is None:
        threshold = max_color_value // 2

    img_pb = []
    for row in img_cinza:
        pb_row = []
        for pixel_value in row:
            # Aplicar o limiar
            if pixel_value >= threshold:
                pb_pixel_value = max_color_value  # Branco
            else:
                pb_pixel_value = 0  # Preto
            pb_row.append(pb_pixel_value)
        img_pb.append(pb_row) # Adicionando a linha à lista de linhas


    return img_pb




In [ ]:
import struct

img_caminho = input("Entre com o caminho da imagem .ppm: ")

def save_img(image_data, width, height, max_color_value, filepath, is_color=False):

    try:
        with open(filepath, 'wb') as f:
            # Escrever o número mágico (P6 para cor, P5 para escala de cinza)
            magic_number = b'P6' if is_color else b'P5'
            f.write(magic_number + b'\n')

            # Escreva as dimensões
            f.write(f"{width} {height}\n".encode())

            # Escrever valor máximo de cor
            f.write(f"{max_color_value}\n".encode())

            # Escrever dados de pixel
            if is_color:
                for row in image_data:
                    for pixel in row:
                        f.write(struct.pack('BBB', pixel[0], pixel[1], pixel[2]))
            else:
                for row in image_data:
                    for pixel_value in row:
                        f.write(struct.pack('B', pixel_value))

        print(f"Imagem salva com sucesso em {filepath}")

    except Exception as e:
        print(f"Um erro ocorreu ao salvar a imagem: {e}")

# Carregando caminho da imagem
image_info = load_img(img_caminho)

if image_info:
    image_data, width, height, max_color_value = image_info

    # Processando imagem para salvar em escala de cinza
    processed_image_data = converte_cinza(image_data, max_color_value)
    output_filepath = "Cinza.ppm"
    save_img(processed_image_data, width, height, max_color_value, output_filepath, is_color=False)

    # Processando imagem para salvar em preto e branco
    processed_image_data = converte_p_b(image_data, max_color_value)
    output_filepath = "Preto_e_Branco.ppm" # Alterado o nome do ficheiro para mais clareza
    save_img(processed_image_data, width, height, max_color_value, output_filepath, is_color=False)
else:
    print("Não foi possível carregar a imagem.")

Entre com o caminho da imagem .ppm: /content/2.ppm
Imagem salva com sucesso em Cinza.ppm
Imagem salva com sucesso em Preto_e_Branco.ppm
